### Making the County Table
This is the table that has the county name, county id, and corresponding state id.
It will be used to pull state-level information into our examples for each county

In [1]:
import pandas as pd
import numpy as np
import time
import requests
import json

In [2]:
# Load all state ids from table
df_state_table = pd.read_csv('state_table.csv')
print(df_state_table.head())
df_county_table = pd.DataFrame()

      id        name
0  27282     Alabama
1  27283      Alaska
2  27284     Arizona
3    149    Arkansas
4  27286  California


In [5]:
# Loop over all state ids to get the list of counties within each state
states_wo_counties = []
no_categories = []

states_list = list(df_state_table['id'].values)
# Alaska, Louisiana, and Virgin Islands dont have counties so we will deal with those later
states_list.remove(27283)
states_list.remove(27300)
states_list.remove(33605)

for state_id in states_list:
    time.sleep(2)
    params = {
        'category_id': state_id,
        'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
        'file_type': 'json'
    }
    r = requests.get(url = 'https://api.stlouisfed.org/fred/category/children', params=params)
    res = r.json()
    try:
        found_counties = False
        if 'categories' in list(res.keys()):
            for child in res['categories']:
                if child['name'] == 'Counties':
                    state_counties_id = child['id']
                    found_counties = True
        else:
            no_categories.append(state_id)


        if found_counties:
            params = {
                'category_id': state_counties_id,
                'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
                'file_type': 'json'
            }
            r = requests.get(url = 'https://api.stlouisfed.org/fred/category/children', params=params)
            res = r.json()
            df_county_table = df_county_table.append(pd.DataFrame(res['categories']))
        else:
            states_wo_counties.append(state_id)
    except:
        print(state_id)
        print(res)

In [6]:
# Print results
df_county_table.reset_index(inplace=True, drop=True)
print(df_county_table.shape)
print(states_wo_counties)
print(no_categories)

(3078, 3)
[]
[]


In [7]:
# Alaska
params = {
    'category_id': 27283,
    'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
    'file_type': 'json'
}
r = requests.get(url = 'https://api.stlouisfed.org/fred/category/children', params=params)
res = r.json()

for child in res['categories']:
    if child['name'] == 'Census Areas and Boroughs':
        state_counties_id = child['id']

params = {
    'category_id': state_counties_id,
    'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
    'file_type': 'json'
}
r = requests.get(url = 'https://api.stlouisfed.org/fred/category/children', params=params)
res = r.json()
df_county_table = df_county_table.append(pd.DataFrame(res['categories']))

In [8]:
# Louisiana
params = {
    'category_id': 27300,
    'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
    'file_type': 'json'
}
r = requests.get(url = 'https://api.stlouisfed.org/fred/category/children', params=params)
res = r.json()

    for child in res['categories']:
        if child['name'] == 'Parishes':
            state_counties_id = child['id']

    params = {
        'category_id': state_counties_id,
        'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
        'file_type': 'json'
    }
    r = requests.get(url = 'https://api.stlouisfed.org/fred/category/children', params=params)
    res = r.json()
    df_county_table = df_county_table.append(pd.DataFrame(res['categories']))

In [9]:
# Virgin Islands
df_county_table.append({'id': 33605, 'name': 'Virgin Islands', 'parent_id': 33605}, ignore_index=True)

id                         name  parent_id
0     27336           Autauga County, AL      27335
1     27337           Baldwin County, AL      27335
2     27338           Barbour County, AL      27335
3     27339              Bibb County, AL      27335
4     27340            Blount County, AL      27335
...     ...                          ...        ...
3236  28522  West Baton Rouge Parish, LA      28461
3237  28523      West Carroll Parish, LA      28461
3238  28524    West Feliciana Parish, LA      28461
3239  28525              Winn Parish, LA      28461
3240  33605               Virgin Islands      33605

[3241 rows x 3 columns]

In [11]:
print(df_county_table.shape)

(3240, 3)


In [12]:
df_county_table.rename(columns={'parent_id': 'state_id', 'id':'county_id'}, inplace=True)

In [14]:
df_county_table.to_csv('county_table.csv', index=False)